# HDFS

## ¿Qué es HDFS?
- Hadoop Distributed File System (HDFS)
    - Sistema de archivos distribuido y tolerante a fallos
    - Alta disponibilidad y altas prestaciones
    - Accesible mediante línea de comandos e interfaz gráfico
    - Escalable
    - Coherencia de datos: WORM (Write Once, Read Many)
    - Portable a diferentes piezas de hardware y software
    
## Arquitectura de HDFS
![02_hadoop_1]

- NameNode
    - El servidor maestro que coordina todo el trabajo del cluster HDFS
    - No almacena datos, solamente metadatos (<bloque, nodo>, ...)
    - Implementa alta disponibilidad mediante un segundo NameNode que permanece en standby y que está al tanto de las actualizaciones del sistema
    
- DataNodes
    - Almacenan los datos
    - Envía heartbeats al NameNode
    - Cuando un DataNode no envía heartbeat, el NameNode lo marca como muerto e inicia la replicación de los datos a otros DataNodes
    
## Escrituras en HDFS
- Un fichero de gran tamaño será divido en unidades denominadas "bloques".
- Cada bloque será copiado en 1-N nodos (factor de replicación)
    - De tal forma que en un nodo podremos tener 1-N "bloques"
- Los DataNodes que almacenarán cada bloque se seleccionan teniendo en cuenta, entre otros aspectos, su localización en la red ("rack-awareness").

## Lecturas en HDFS
- Se pregunta desde cliente dónde está el fichero X al NameNode
- El NameNode reconoce que este fichero está dividido en N bloques repartidos en N nodos
- El namenode devuelve el fichero
- Para seleccionar el DataNode del que leer, se elege el más cercano
- Para ello se calculan las distancias, donde existen los siguientes niveles:
    - Diferentes procesos en el mismo nodo
    - Diferentes nodos en el mismo rack
    - Nodos en distintos racks del mismo datacenter
    - Nodos en diferentes datacenters
- Por ejemplo, sea el nodo n1, del rack r1, del datacenter d1 (representado como /d1/r1/n1). En este escenario:
    - distancia (/d1/r1/n1, /d1/r1/n1) = 0 (procesos en el mismo nodo)
    - distancia (/d1/r1/n1, /d1/r2/n2) = 2 (diferentes nodos del mismo rack)
    - distancia (/d1/r1/n1, /d1/r3/n3) = 4 (nodos en diferentes racks del mismo datacenter)
    - distancia (/d1/r1/n1, /d2/r3/n4) = 6 (nodos en diferentes datacenters)
    
## HDFS: rack-awareness
- ¿Qué es un rack-awareness?
    - Los rack son armarios donde se encuentran los servidores instalados
    - Los servidores del mismo rack comparten una serie de servicios
        - Switch de conexión a la red, alimentación, ventilación, ...
    - Los fallos de esos servicios afectan a todos los servidores del rack
    
## HDFS: Resumen
- HDFS explota el paralelismo del sistema para leer los ficheros con mayor rapidez (Write Once Read Many)
- HDFS divide los ficheros en bloques de gran tamaño (normalmente 64,128M)
    - Menor tamaño congestionaría el NameNode
- Mejor pocos grandes ficheros
- Formas de insertar datos en HDFS:
    - Copiarlos manualmente
    - Flume
    - Sqoop
    - ...
    
## HDFS: Problemas de la versión 1
- En su versión 1.0, HDFS tenía los siguientes inconvenientes
    - El namenode es único. Esto imnplica una serie de limitaciones entre las que destacan las siguientes:
        - Número de ficheros almacenados. Esto se debe a que el NameNode mantiene los metadatos en memoria.
        - Espacio de nombres único: esto provoca que el NameNode no puede delegar carga de trabajo por lo que se convierte en un cuello de botella
        - Single Point of Failure (SPOF)
    - Diseñado solamente para ejecutar aplicaciones MapReduce

## HDFS: Federación de Namenodes
- Para afrontar estos problemas, se propone la federación de NameNodes
    - Permite tener espacios de nombres independientes
    - Se elimina el SPOF
    - Se permte mayor número de ficheros almacenados

# MapReduce

## ¿Qué es MapReduce?
- MapReduce es un modelo de programación paralela distribuida enfocado a grandes conjuntos de datos procesados en un cluster
- Automatiza la paralelización de las ejecuciones así como la distribución de las tareas entre los nodos
- Tiene varias fases
    - Map
        - Opera en un único bloque de un fichero HDFS
        - Se ejecuta siempre que sea posible en el nodo que almacena dicho bloque, lo que se minimiza el tráfico sobre la red.
        - Salida: pares <clave, valor> -> Es importante seleccionar adecuadamente estos pares.
    - Shuffle & sort
        - Ordena y consolida los datos intermedios de todos los maps
        - Sucede cuando todas las tareas map han acabado y antes de que comiencen las tareas reduce
    - Reduce
        - Opera sobre los resultados intermedios ordenados y barajados (la salida de las tareas map)
        - Produce los resultados finales

- MapReduce trata de minimizar las transferencias de información sobre la red para mejorar las prestaciones del sistema
    - Los maps se ejecutan, en la medida de lo posible, en el mismo nodo que almacena el bloque
- Automatiza la diseminación de las tareas a través de los nodos, la gestión de los fallos, ... de forma que el usuario solamente se tiene que concentrar en las funciones map y reduce

## Comparación entre MapReduce y una Base de datos Distribuida

|MapReduce + HDFS|BD distribuida|
|---|---|
|Ejecuta aplicaciones arbitrarias|Ejecución paralela de consultas SQL sobre un cluster|
|Tipos de datos y formatos de almacenamientos variados|Esquema de datos y formato de almacenamiento  redefinido|
|«Esquema» definido al leer|Esquema definido al escribir|
|Tolera fallos, no reinicia tareas por un fallo|No tolera fallos, reinicia tareas por un fallo|
|Extensible, se pueden implementar modelos de programación sobre él|No extensible, solamente soportan SQL|

## Hadoop Streaming
- Hadoop Streaming es una utilidad que se distribuye junto con MapReduce
- Utiliza las entradas/salidas estándar de Unix (stdin & stdout)
- Cuando Hadoop Streaming ejecuta un trabajo, cada tarea map se ejecuta en su propio proceso utilizando el ejecutable que se proporciona en la orden
- Seguidamente, los ficheros de entrada se convierten en líneas de texto y se redireccionan a la entrada estándar del map
- La salida map son pares <clave, valor>, donde las claves y valores se encuentran separados por un separador (por defecto, el tabulador)
- Los reducers también se ejecutan en su propio proceso utilizando su propio ejecutable
- La salida del map se ordena y se mezcla de forma que los pares <clave, valor> con la misma clave vayan al mismo reducer
- La salida del reducer se redirecciona a stdout
- Por tanto, para redactar tabajos MapReduce en Python utilizando Hadoop Streaming, hay que preparar dos ficheros con código Python: la función map y la función reduce
- Ejemplo de contador de palabras en Python:
[02_hadoop_1]:images/02_hadoop_1.png

In [2]:
%%writefile mapper.py
import sys

#entrada de la entrada estandar STDIN
for line insys.stdin:
    # eliminamos espacios blancos al principio y final
    line = line.trip()
    # dividimos la linea en palabras
    words = line.split()
    # incrementamos los contadores
    for word in words:
        # escribimos los resultados a la salida estandar STDOUT
        # Esta salida será la entrada para el reduce, es decir, par reducer01.py
        # Delimitado por tab, para cada palabra ponemos 1 ocurrencia
        print '%s\t%s'(word,1)

Writing mapper.py


In [4]:
%%writefile reducer.py
from operator import itemgetter
import sys

current_word=None
current_count=0
word=None

# entrada desde STDIN
for line in sys.stdin:
    # eliminamos espacios blancos al principio y final
    line = line.strip()
    # pareamos la entrada que hemos obtenido del mapper01.py
    word, count = line.split('\t',1)
    
    try:
        count = int(count)
    except ValueError:
        continue
    
    if current_word == word:
        current_count += count
        else:
            if current_word
                print '%s\t%s' % (current_word, current_count)
            current_count = count
            current_word = word
    if current_word == word:
        print '%s\t%s' % (current_word, current_count)

Overwriting reducer.py


- Ejecutándolo en local (no usamos Hadoop)
    - Creamos en local una carpeta llamada examplemapreduce y dejamos en ella los ficheros mapper01.py y reducer01.py
    - Creamos un fichero de texto en esta carpeta, fichero.txt
    > echo "Hola esto es una prueba de un contador de palabras" > fichero.txt
    - Desde esa carpeta, ejecutamos:
    > cat fichero.txt | ./mapper01.py | sort | ./reducer.py

- Ejecutándolo en Hadoop:
    - Partiendo de la carpeta y fichero de texto creados, creamos en HDFS una carpeta llamada examplemapreduce y cargamos en ella el fichero de texto
        > hdfs dfs -mkdir examplemapreduce
        >
        > hdfs dfs -put fichero.txt examplemapreduce
    - Ejecutamos el trabajo mapreduce de la siguiente forma;
        > hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar
        > 
        > -file mapper01.py, reducer01.py -mapper mapper01.py -reduce reducer01.py
        >
        > -input /path/to/examplemapreduce/*
        >
        > -output /path/to/examplemapreduce-output

## Aspectos de diseño
- Al diseñar programas MapReduce es necesario tener en cuenta algunos aspectos de diseño
    - Utilizar un ``combiner``, entre el map y el reduce
    - Elegir de forma apropiada las claves de los pares <clave, valor>
    - Dedicir de forma inteligente dónde (en el map o el reduce) se implementa cada funcionalidad
    - Tener en cuenta la localidad de los cálculos
    
## Combiner
- Los map producen una gran cantidad de datos de salida que se tienen que enviuar por la red, barajar, oredenar y reducir
    - Esto consume altas cantidades de ancho de banda de red, así como recuros (memoria, CPU) en el reducer
- Para reducir el consumo de recursos, se utilizan los ``combinadores`` 
- Estos realizan cómputos sobre las salidas del map, en los nodos donde se ejecutan los map, antes de que se envíen por la red al reducer adecuado
- Normalmente, el código del combiner es el mismo (o muy similar) al reducer

## Partitioner
- Normalmente las claves salida del map se envían a un reducer de forma aleatoria (los pares <clave, valor> con la misma clave van al mimso reducer
    - ej.: Todos los pares <perro,1> van al reducer 1, todos los pares <gato, 1> van al reducer 2, etc
- El problema viene cuando las claves no están equilibradas (hay muchos pares <clave,valor> con una determinada clave y muy pocos con otros).
     - La mayor parte de los reducer estarán infrautilizados, mientras que el que reciba la clave popular estará sobrecargado
- Es posible implementar un ``particionador`` que utilice alguna regla para dividir el espacio de valores de la clave, de forma que la carga de trabajo se reparta de forma más equitativa entre los reducers

## Localidad de los cálculos
- Map
    - Puede haber tantos procesos map como nodos en el cluster
    - Los map se ejecutan en la medida de lo posible, en los nodos del cluster que almacenan los datos de entrada
- Reduce
    - Puede haber tantos procesos reduce como claves distintas en la salida del map
    - Los reduce se ejecutan en cualquier nodo del cluster
        - Un reducer recibe todos los pares <clave, valor> para la misma clave
    - Es recomendable poner la complejidad en el map y dejar el reduce lo más sencillo posible, para explotar el paralelismo

## Patrones de diseño
- Los patrones de diseño muestran estrategias genéricas que pueden ser de utilidad a la hora de afrontar un problema dado
- Los principales patrones se pueden agrupar en las siguientes categorías
    - Resúmenes
        - Agrupa los datos de entrada en función de algún parámetro (hora, día y hora, usuario, ...) y calculan alguna característica de interés
        - Los problemas que se resuelven con este patrón son:
            - Resúmenes numéricos: conteos, máximo, mínimo, media, ...
![02_hadoop_3]
            - Índices invertidos: es importante cuando estamos desarrollando una herramienta de búsqueda para una nueva aplicación
                - Ej.: en qué página se menciona una palabra
![02_hadoop_4]
    - Filtrado
        - Crea subconjuntos de los datos basándose en un criterio sin modificar los datos originales
            - Ej.: muestra los resultados del último año en un dataset que contiene datos de 10 años
            - Tipos de filtrados:
                 - Filtrado simple
![02_hadoop_5]
                 - Filtro bloom
![02_hadoop_6]
                 - Top N
![02_hadoop_7]
                 - Muestreo aleatorio
    - Reuniones (joins)
        - Es útil cuando tenemos distintos datasets que deseamos reunir utilizando una clave
            - Operaciones de conjuntos
![02_hadoop_8]
            - Estructura del patrón reunión
![02_hadoop_9]

[02_hadoop_3]:images/02_hadoop_3.png
[02_hadoop_4]:images/02_hadoop_4.png
[02_hadoop_5]:images/02_hadoop_5.png
[02_hadoop_6]:images/02_hadoop_6.png
[02_hadoop_7]:images/02_hadoop_7.png
[02_hadoop_8]:images/02_hadoop_8.png
[02_hadoop_9]:images/02_hadoop_9.png

## MRJob

- Es una librería de Python que permite la programación de trabajos MapReduce
- Permite la ejecución de dichos protgramas MapReduce tanto localmente, como en cluster, como en una variedad de servicios en la Nube (Amazon / Google)
- Se basa en Hadoop Streaming para la ejecución de trabajos

- Ventajas
    - El código de Python no cambia sin importar que estemos trabajando en local, hadoop o nube
    - Su ejecución local permite depurar el código de forma más sencilla que si se ejecuta en un cluster
    - Amplia documentación, desarrollos open-source actualizados

## MRJob: protocolos de entrada, salida e interno

- Los datos que entran en el map, pasan del map al reduce y salen del reduce, siguen un protocolo que define la forma en que se codifican tales datos
- Por defecto
    - INPUT_PROTOCOL = RawValueProtocol
    - INTERNAL_PROTOCOL = JSONProtocol
    - OUTPUT_PROTOCOL = JSONProtocol
- Otras opciones
    - ReprProtocol: Representa cada par <clave, valor> como una representación imprimible
    - PickleProtocol: Utiliza la función pickle de Python para representrar los pares <clave, valor>
- MRJob permite  que en la llamada se incluyan parámetros de entrada
- Se puede realizar una ordenación secundaria
    - Consiste en que los reducers reciban los pares <clave, valor> se encuentren ordenados por valor
    - Para activarla
        > SORT_VALUES = True